# Chapter 3: Data Structures

## Stacks, Queues, and Lists

### 3-3
Give an algorithm to reverse the direction of a given singly linked list. In
other words, after the reversal all pointers should now point backwards. Your
algorithm should take linear time.

In [1]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def reverse(self):
        current = self.head
        prev = None
        while current is not None:
            next = current.next
            current.next = prev
            prev = current
            current = next
        self.head = prev

## Trees and Other Dictionary Structures

### 3-11
Design a dictionary data structure in which search, insertion, and deletion
can all be processed in $O(1)$ time in the worst case. You may assume the set
elements are integers drawn from a finite set $1, 2, ..,\ n$, and initialization can take
$O(n)$ time.

In [2]:
class Dictionary:
    def __init__(self, elements, n):
        self.array = [0] * (n + 1)
        self.max_value = n
        for num in elements:
            self.array[num] = 1

    def search(self, i):
        self.__check_boundaries(i)
        return bool(self.array[i])

    def insert(self, i):
        self.__check_boundaries(i)
        self.array[i] = 1

    def delete(self, i):
        self.__check_boundaries(i)
        self.array[i] = 0

    def __check_boundaries(self, i):
        if i < 1 or i > self.max_value:
            raise ValueError("Number is outside of allowed range")


## Applications of Tree Structures

In [3]:
class TreeNode:
    def __init__(self, data, parent = None):
        self.data = data
        self.parent = parent
        self.left = None
        self.right = None

class Tree:
    def __init__(self, root):
        self.root = root
    
    def insert(self, data):
        new_node = TreeNode(data)
        if self.root is None:
            self.root = new_node
        else:
            self.__insert(self.root, new_node)

    def __insert(self, current_node, new_node):
        if new_node.data < current_node.data:
            if current_node.left is None:
                current_node.left = new_node
                new_node.parent = current_node
            else:
                self.__insert(current_node.left, new_node)
        else:
            if current_node.right is None:
                current_node.right = new_node
                new_node.parent = current_node
            else:
                self.__insert(current_node.right, new_node)
        # and readjust it to be a properly balanced binary search tree

    def delete(self, node):
        if node is None:
            return
    
        if node.left is None and node.right is None:
            # Node has no children
            if node.parent:
                if node.parent.left is node:
                    node.parent.left = None
                else:
                    node.parent.right = None
            else:
                # This is the root node with no children
                self.root = None
        elif node.left is None or node.right is None:
            # Node has one child
            child = node.left if node.left else node.right
            if node.parent:
                if node.parent.left is node:
                    node.parent.left = child
                else:
                    node.parent.right = child
            else:
                # This is the root node with one child
                self.root = child
        else:
            # Node has two children
            successor = self.find_successor(node)
            node.data = successor.data
            self.delete(successor)
        # and readjust it to be a properly balanced binary search tree
    
    def find_successor(self, node):
        successor = node.right
        while successor.left:
            successor = successor.left
        return successor

    def get_size(self):
        if self.root is None:
            return 0
        return self.__get_size(self.root)
    
    def __get_size(self, node):
        left_size = right_size = 0
        if node.left:
            left_size = self.__get_size(node.left)
        if node.right:
            right_size = self.__get_size(node.right)
        return 1 + left_size + right_size

### 3-25
In the *bin-packing* problem, we are given $n$ objects, each weighing at most 1 kilogram. Our goal is to find the smallest number of bins that will hold the $n$ objects, with each bin holding 1 kilogram at most.
* The *best-fit heuristic* for bin packing is as follows. Consider the objects in the order in which they are given. For each object, place it into the partially filled bin with the *smallest* amount of extra room after the object is inserted. If no such bin exists, start a new bin. Design an algorithm that implements the best-fit heuristic (taking as input the $n$ weights $w_1, w_2, ..., w_n\ $ and outputting the number of bins used) in $O(n \log{} n)$ time.
* Repeat the above using the *worst-fit heuristic*, where we put the next object into the partially filled bin with the *largest* amount of extra room after the object is inserted.

In [4]:
def pack_bin(objects, method):
    # Use a balanced binary search tree in which the node data is the space left in a given bin
    MAX_WEIGHT = 1000 # Floating point calculations can give incorrect results, better to use integers (1000g = 1kg)
    tree = Tree(None)

    for weight in objects:
        if weight == 0:
            continue
        node = method(tree.root, weight)
        if node is None:
            tree.insert(MAX_WEIGHT - weight)
        else:
            new_data = node.data - weight
            tree.delete(node)
            tree.insert(new_data)

    return tree.get_size()

def find_best_fit(node, weight):
    while node is not None and weight > node.data:
        node = node.right
    return node

def find_worst_fit(node, weight):
    # Find node with max value
    if node is None:
        return
    max_node = node
    while max_node.right is not None:
        max_node = max_node.right
    return max_node if max_node.data >= weight else None

### 3-26
Suppose that we are given a sequence of $n$ values $x_1, x_2, ..., x_n\ $ and seek to quickly answer repeated queries of the form: given $i$ and $j$, find the smallest value in $x_i, . . . , x_j\ $.
* Design a data structure that uses $O(n^2)$ space and answers queries in $O(1)$ time.
* Design a data structure that uses $O(n)$ space and answers queries in $O(\log{n})$ time. For partial credit, your data structure can use $O(n \log{n} )$ space and have $O(\log{n})$ query time.

In [5]:
def build_tree_structure(sequence, start = None, end = None):
    size = len(sequence)
    if start is None or end is None:
        start = 0
        end = size - 1

    min_element = min(sequence)
    node = TreeNode({"start": start, "end": end, "min": min_element})
    
    if size > 1:
        midpoint = size//2
        node.left = build_tree_structure(sequence[:midpoint], start, start + midpoint - 1)
        node.right = build_tree_structure(sequence[midpoint:], start + midpoint, end)
        
    return node

def query_tree(node, i, j):
    if node is None:
        return
    
    if i > j:
        i, j = j, i

    start = node.data["start"]
    end = node.data["end"]
    if i <= start and j >= end:
        return node.data["min"]

    return tree_min(query_tree(node.left, i, j), \
               query_tree(node.right, i, j))

def tree_min (a, b):
    if a is None:
        return b
    if b is None:
        return a
    return min(a, b)

def build_brute_force_data_structure(sequence):
    matrix = []
    
    for i in range(len(sequence)):
        mins = []
        min = sequence[i]
        for j in range(i, len(sequence)):
            if sequence[j] < min:
                min = sequence[j]
            mins.append(min)
        matrix.append(mins)

    return matrix

def query_matrix(matrix, i, j):
    if i < j:
        return matrix[i][j-i]
    return matrix[j][i-j]